In [1]:
%pip install pandas numpy matplotlib seaborn scikit-learn kagglehub

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder


/home/diggspapu/lab01_MLE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Download latest version
path = kagglehub.dataset_download("marusagar/bank-transaction-fraud-detection")

print("Path to dataset files:", path)

Path to dataset files: /home/diggspapu/.cache/kagglehub/datasets/marusagar/bank-transaction-fraud-detection/versions/1


In [4]:
# Find the CSV file in the dataset directory
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
dataset_file = os.path.join(path, csv_files[0])

# Load the dataset into a DataFrame
data = pd.read_csv(dataset_file)
data.head()

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,...,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,23-01-2025,...,Restaurant,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",POS,0,INR,+9198579XXXXXX,Bitcoin transaction,oshaXXXXX@XXXXX.com
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,11-01-2025,...,Restaurant,74622.66,POS Mobile Device,"Nashik, Maharashtra",Desktop,0,INR,+9191074XXXXXX,Grocery delivery,hredhaanXXXX@XXXXXX.com
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,25-01-2025,...,Groceries,66817.99,ATM,"Bhagalpur, Bihar",Desktop,0,INR,+9197745XXXXXX,Mutual fund investment,ekaniXXX@XXXXXX.com
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,19-01-2025,...,Entertainment,58177.08,POS Mobile App,"Chennai, Tamil Nadu",Mobile,0,INR,+9195889XXXXXX,Food delivery,yaminiXXXXX@XXXXXXX.com
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,30-01-2025,...,Entertainment,16108.56,Virtual Card,"Amritsar, Punjab",Mobile,0,INR,+9195316XXXXXX,Debt repayment,kritikaXXXX@XXXXXX.com


## Transformation

In [5]:
print(data.columns)
print(data.shape)
print(data.info())
print(data.select_dtypes(exclude=np.number).columns)
print(data.describe(include='all'))

Index(['Customer_ID', 'Customer_Name', 'Gender', 'Age', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Merchant_ID',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Transaction_Location', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency', 'Customer_Contact', 'Transaction_Description',
       'Customer_Email'],
      dtype='object')
(200000, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Customer_ID              200000 non-null  object 
 1   Customer_Name            200000 non-null  object 
 2   Gender                   200000 non-null  object 
 3   Age                      200000 non-null  int64  
 4   State                    200000 non-null  object 
 5   

### Standarize strings

In [6]:
number_cols = data.select_dtypes(include=[np.number]).columns
string_cols = data.select_dtypes(exclude=[np.number]).columns
for col in string_cols:
    data[col] = data[col].apply(lambda x: x.strip().lower() if isinstance(x, str) else x)

### Handle dates

In [7]:
data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'], errors='coerce')
data['Transaction_Time'] = pd.to_datetime(data['Transaction_Time'], format='%H:%M:%S', errors='coerce').dt.time
data['Transaction_Datetime'] = pd.to_datetime(data['Transaction_Date'].astype(str) + ' ' + data['Transaction_Time'].astype(str), errors='coerce')

/tmp/ipykernel_7189/2324014094.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'], errors='coerce')


### Handle Nulls

In [8]:
data.isna().sum()


Customer_ID                0
Customer_Name              0
Gender                     0
Age                        0
State                      0
City                       0
Bank_Branch                0
Account_Type               0
Transaction_ID             0
Transaction_Date           0
Transaction_Time           0
Transaction_Amount         0
Merchant_ID                0
Transaction_Type           0
Merchant_Category          0
Account_Balance            0
Transaction_Device         0
Transaction_Location       0
Device_Type                0
Is_Fraud                   0
Transaction_Currency       0
Customer_Contact           0
Transaction_Description    0
Customer_Email             0
Transaction_Datetime       0
dtype: int64

Aquí se haría un procedimiento para manejar nulos, en este caso no aplica pero igual

In [9]:
for col in data.columns:
    if data[col].isnull().sum() > 0:
        if data[col].dtype in ['object', 'category']:
            mode = data[col].mode()[0]
            data[col].fillna(mode, inplace=True)
        elif np.issubdtype(data[col].dtype, np.number):
            median = data[col].median()
            data[col].fillna(median, inplace=True)
        elif 'datetime' in str(data[col].dtype):
            data[col].fillna(method='ffill', inplace=True)

### Encoding

In [10]:
string_cols

Index(['Customer_ID', 'Customer_Name', 'Gender', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Merchant_ID', 'Transaction_Type',
       'Merchant_Category', 'Transaction_Device', 'Transaction_Location',
       'Device_Type', 'Transaction_Currency', 'Customer_Contact',
       'Transaction_Description', 'Customer_Email'],
      dtype='object')

In [11]:
le = LabelEncoder()
data['Gender_Encoding'] = data['Gender'].map({'male': 1, 'female': 0}).fillna(-1)
data['Bank_Branch_Encoding'] = le.fit_transform(data['Bank_Branch'])


In [12]:
data[number_cols].head()

,Age,Transaction_Amount,Account_Balance,Is_Fraud
0,60,32415.45,74557.27,0
1,51,43622.60,74622.66,0
2,20,63062.56,66817.99,0
3,57,14000.72,58177.08,0
4,43,18335.16,16108.56,0


In [13]:
data[string_cols].head()

,Customer_ID,Customer_Name,Gender,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,Transaction_Time,Merchant_ID,Transaction_Type,Merchant_Category,Transaction_Device,Transaction_Location,Device_Type,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,osha tella,male,kerala,thiruvananthapuram,thiruvananthapuram branch,savings,4fa3208f-9e23-42dc-b330-844829d0c12c,2025-01-23,16:04:07,214e03c5-5c34-40d1-a66c-f440aa2bbd02,transfer,restaurant,voice assistant,"thiruvananthapuram, kerala",pos,inr,+9198579xxxxxx,bitcoin transaction,oshaxxxxx@xxxxx.com
1,7c14ad51-781a-4db9-b7bd-67439c175262,hredhaan khosla,female,maharashtra,nashik,nashik branch,business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,2025-01-11,17:14:53,f9e3f11f-28d3-4199-b0ca-f225a155ede6,bill payment,restaurant,pos mobile device,"nashik, maharashtra",desktop,inr,+9191074xxxxxx,grocery delivery,hredhaanxxxx@xxxxxx.com
2,3a73a0e5-d4da-45aa-85f3-528413900a35,ekani nazareth,male,bihar,bhagalpur,bhagalpur branch,savings,e41c55f9-c016-4ff3-872b-cae72467c75c,2025-01-25,03:09:52,97977d83-5486-4510-af1c-8dada3e1cfa0,bill payment,groceries,atm,"bhagalpur, bihar",desktop,inr,+9197745xxxxxx,mutual fund investment,ekanixxx@xxxxxx.com
3,7902f4ef-9050-4a79-857d-9c2ea3181940,yamini ramachandran,female,tamil nadu,chennai,chennai branch,business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,2025-01-19,12:27:02,f45cd6b3-5092-44d0-8afb-490894605184,debit,entertainment,pos mobile app,"chennai, tamil nadu",mobile,inr,+9195889xxxxxx,food delivery,yaminixxxxx@xxxxxxx.com
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,kritika rege,female,punjab,amritsar,amritsar branch,savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,2025-01-30,18:30:46,70dd77dd-3b00-4b2c-8ebc-cfb8af5f6741,transfer,entertainment,virtual card,"amritsar, punjab",mobile,inr,+9195316xxxxxx,debt repayment,kritikaxxxx@xxxxxx.com


### Drop unnecessary features

In [14]:
data.columns

Index(['Customer_ID', 'Customer_Name', 'Gender', 'Age', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Merchant_ID',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Transaction_Location', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency', 'Customer_Contact', 'Transaction_Description',
       'Customer_Email', 'Transaction_Datetime', 'Gender_Encoding',
       'Bank_Branch_Encoding'],
      dtype='object')

In [15]:
data_regression = data.copy()
data_regression = data_regression[['Transaction_Amount', 'Account_Balance', 'Is_Fraud']]
data_classification = data.copy()
data_classification.drop(columns=[
    'Customer_ID', 'Customer_Name', 'City', 'Bank_Branch', "Customer_Email", "Transaction_ID", "Merchant_ID", 
    "Customer_Contact", "Transaction_Currency", "Device_Type", "Transaction_Location", "Transaction_Description"
    ], inplace=True)

In [16]:
data_regression
data_classification

,Gender,Age,State,Account_Type,Transaction_Date,Transaction_Time,Transaction_Amount,Transaction_Type,Merchant_Category,Account_Balance,Transaction_Device,Is_Fraud,Transaction_Datetime,Gender_Encoding,Bank_Branch_Encoding
0,male,60,kerala,savings,2025-01-23,16:04:07,32415.45,transfer,restaurant,74557.27,voice assistant,0,2025-01-23 16:04:07,1,127
1,female,51,maharashtra,business,2025-01-11,17:14:53,43622.60,bill payment,restaurant,74622.66,pos mobile device,0,2025-01-11 17:14:53,0,100
2,male,20,bihar,savings,2025-01-25,03:09:52,63062.56,bill payment,groceries,66817.99,atm,0,2025-01-25 03:09:52,1,13
3,female,57,tamil nadu,business,2025-01-19,12:27:02,14000.72,debit,entertainment,58177.08,pos mobile app,0,2025-01-19 12:27:02,0,22
4,female,43,punjab,savings,2025-01-30,18:30:46,18335.16,transfer,entertainment,16108.56,virtual card,0,2025-01-30 18:30:46,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,female,55,dadra and nagar haveli and daman and diu,business,2025-01-08,18:42:09,98513.74,credit,restaurant,37475.11,desktop/laptop,0,2025-01-08 18:42:09,0,33
199996,male,51,manipur,business,2025-01-01,20:51:21,40593.55,withdrawal,groceries,53037.20,atm,0,2025-01-01 20:51:21,1,63
199997,female,41,chandigarh,savings,2025-01-28,10:47:40,61579.70,withdrawal,health,96225.36,atm,0,2025-01-28 10:47:40,0,21
199998,female,28,telangana,checking,2025-01-08,06:26:41,39488.22,debit,electronics,89599.90,voice assistant,0,2025-01-08 06:26:41,0,103


### Feature Engineering

In [ ]:
# Crear nuevas features
data_classification['Transaction_Hour'] = data_classification['Transaction_Time'].apply(lambda x: x.hour if pd.notnull(x) else np.nan)
data_classification['Is_Night'] = data_classification['Transaction_Hour'].apply(lambda x: 1 if x >= 22 or x < 6 else 0)
data_classification['Transaction_Weekday'] = data_classification['Transaction_Date'].dt.day
data_classification['Transaction_Weekend'] = data_classification['Transaction_Weekday'].apply(lambda x: 1 if x in [5, 6] else 0)
data_classification.drop(columns="Transaction_Weekday", inplace=True)
data_regression['Transaction_Amount_to_Balance'] = data_regression['Transaction_Amount'] / (data_regression['Account_Balance'])
# Normalizar
scaler = MinMaxScaler()
data_regression["Transaction_Amount_Scaler"] = scaler.fit_transform(data_regression[["Transaction_Amount"]])

In [19]:
data_regression

,Transaction_Amount,Account_Balance,Is_Fraud,Transaction_Amount_to_Balance,Transaction_Amount_Scaler,Account_Balance_Scaler
0,32415.45,74557.27,0,0.434772,0.327359,0.732180
1,43622.60,74622.66,0,0.584576,0.440574,0.732868
2,63062.56,66817.99,0,0.943796,0.636958,0.650713
3,14000.72,58177.08,0,0.240657,0.141332,0.559755
4,18335.16,16108.56,0,1.138225,0.185119,0.116925
...,...,...,...,...,...,...
199995,98513.74,37475.11,0,2.628778,0.995088,0.341838
199996,40593.55,53037.20,0,0.765379,0.409975,0.505651
199997,61579.70,96225.36,0,0.639953,0.621978,0.960267
199998,39488.22,89599.90,0,0.440717,0.398809,0.890525
